In [357]:
import pandas as pd
import plotly.express as px
import os

min_txs_per_day = 100

In [358]:
df = pd.read_csv('csv_exports/project_deploys_2022_12_09.csv')

df = df[ (df['txs_per_day'] >= min_txs_per_day)]
# df = df[ (df['type'] == 'Contract Project')]
df['year'] = df['launch_time'].str[:4]
df['date'] = pd.to_datetime( df['launch_time'].str[:10] )

df.sample(10)

,project,launch_time,num_contracts,type,num_transactions,txs_per_day,time_elapsed,year,date
164,Op,2022-04-26T13:30:36Z,6,Contract Project,1047796.0,4608.858611,227.343924,2022,2022-04-26
57,0x,2021-12-22T05:13:01Z,51,Contract Project,214240.0,607.446549,352.689468,2021,2021-12-22
104,Rubicon,2021-08-16T19:18:41Z,150,Contract Project,942330.0,1962.769597,480.102199,2021,2021-08-16
183,Gnosis Safe,2021-11-17T11:17:03Z,17270,Contract Project,71350.0,184.159131,387.436667,2021,2021-11-17
193,Socket,2021-12-20T09:43:11Z,34,Contract Project,279258.0,787.747648,354.501852,2021,2021-12-20
197,Kyberswap,2022-06-16T07:54:02Z,67,Contract Project,20305.0,114.991903,176.577650,2022,2022-06-16
243,Ecc Domains,2022-08-10T09:44:58Z,2,Contract Project,15655.0,128.847086,121.500613,2022,2022-08-10
103,Zerion,2022-03-09T12:56:29Z,1,Contract Project,35112.0,127.509547,275.367616,2022,2022-03-09
144,Velodrome,2022-06-01T20:01:23Z,1812,Contract Project,2690760.0,14082.399864,191.072546,2022,2022-06-01
61,Early Optimists,2022-08-31T17:19:17Z,1,NFT Only Project,51450.0,513.549339,100.185116,2022,2022-08-31


In [359]:
df['type'] = df['type'].str.replace('Contract Project','dApp')
df['type'] = df['type'].str.replace('NFT Only Project','NFT Project')

In [360]:
exclude_list = ['Op','Optimistic Explorer']

df = df[~df['project'].isin(exclude_list)]

In [361]:
print(df.groupby(by='year').count())
df = df[df['year'] == '2022']
# display(df)

      project  launch_time  num_contracts  type  num_transactions  \
year                                                                
2021       25           25             25    25                25   
2022       55           55             55    55                55   

      txs_per_day  time_elapsed  date  
year                                   
2021           25            25    25  
2022           55            55    55  


In [362]:
fig = px.scatter(df, x="date", y="txs_per_day",
	        #  size="pop", 
                 color="type",
                 hover_name="project",
                #  size_max=60
                labels={"date":"Deployment Date", "txs_per_day":"Transactions per Day", "type":"App Type"}
                ,title = "Projects Deployed on Optimism in 2022 \
                        <br><sup>Plot Greater Than 100 Transactions per Day</sup>"
                , width=800, height=400
                 )
project_annotations = ['Velodrome','Pooltogether','Galxe','Aave','Curve','Beethoven X','Kwenta','Opensea','Atlantis World']
for p in project_annotations:
        fig.add_annotation(x=df[df['project'] == p]['date'].iloc[0], y=df[df['project'] == p]['txs_per_day'].iloc[0],
                text=p,
                showarrow=False,
                yshift=12,
                bgcolor="#E6ECF5",
                opacity=0.9
                )
fig.show()
if not os.path.exists("img_outputs"):
        os.mkdir("img_outputs")
fig.write_image("img_outputs/op_deployments_2022.svg")
fig.write_image("img_outputs/op_deployments_2022.png")